In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Algoritmo de confiabilidade FOSM (First Order Second Moment)

In [2]:
def g_x(xVar):
    valor_g_x =  xVar[0, 0] * xVar[0, 1] - xVar[0, 2]
    return valor_g_x

def g_y(yVar):
    valor_g_y =  12.5 * yVar[0, 0] * yVar[0, 1] + 100 * yVar[0, 1] + 250 * yVar[0, 1] - 200 * yVar[0, 2] + 1000 
    return valor_g_y

def HRLF(diff_g_y, norma_g_y, valor_g_y, yVar):
    alfa = diff_g_y/norma_g_y
    beta = np.linalg.norm(yVar)
    yVarNew = -alfa *(beta + valor_g_y/norma_g_y)
    return yVarNew, alfa, beta

    
def jacobiano_g_x(xVar):
    g1_x = xVar[0, 1]
    g2_x = xVar[0, 0]
    g3_x = -1
    diff_g_x = np.array([[g1_x, g2_x, g3_x]]) 
    return diff_g_x

def jacobiano_g_y(yVar):
    g1_y = 12.5 * yVar[0, 1] + 250
    g2_y = 12.5 * yVar[0, 0] + 100
    g3_y = -200
    diff_g_y = np.array([[g1_y, g2_y, g3_y]]) 
    return diff_g_y

def criando_y_var(xVar, global_x, global_num_var, global_jacobiana):
    media_xVar = np.zeros((1,3))
    for local_i_count in range(global_num_var):
        media_xVar[0, local_i_count] = global_x[local_i_count, 0]
    termo_1 = xVar - media_xVar
    termo_2 = np.linalg.inv(global_jacobiana)
    y_var = np.dot(termo_1,termo_2)
    return y_var

def criando_x_var(yVar, global_jacobiana, global_num_var, global_x):
    media_xVar = np.zeros((1,3))
    for local_i_count in range(global_num_var):
        media_xVar[0, local_i_count] = global_x[local_i_count, 0]
    xVar = np.dot(global_jacobiana, y_var.T) + media_xVar.T
    return xVar

global_num_var = 3
global_x = np.array([[40, 5,  0], [50, 2.5,  0], [1000, 200,  0]])

In [3]:
global_jacobiana=np.eye(global_num_var, dtype=float)

In [4]:
for global_i_count in range(global_num_var):
    global_jacobiana[global_i_count, global_i_count] = global_jacobiana[global_i_count, global_i_count]*global_x[global_i_count, 1]
global_jacobiana

array([[  5. ,   0. ,   0. ],
       [  0. ,   2.5,   0. ],
       [  0. ,   0. , 200. ]])

Início do algoritmo FOSM

In [6]:
iteracao = 5
xVar = np.array([[40, 50 ,1000]])
yVar = criando_y_var(xVar, global_x, global_num_var, global_jacobiana)
beta = np.linalg.norm(yVar)
resultados = np.zeros((iteracao,6))
for i in range(iteracao):
    resultados[i, 0] = i
    valor_g_x = g_x(xVar)
    valor_g_y = g_y(yVar)
    diff_g_y = jacobiano_g_y(yVar)
    norma_diff_gy = np.linalg.norm(diff_g_y)
    yVarNew, alfa, beta = HRLF(diff_g_y, norma_diff_gy, valor_g_y, yVar)
    resultados[i, 1] = beta
    resultados[i, 2] = yVarNew[0,0]
    resultados[i, 3] = yVarNew[0,1]
    resultados[i, 4] = yVarNew[0,2]
    betaNew = np.linalg.norm(yVarNew)
    resultados[i, 5] = betaNew
    yVar = yVarNew
resultados = pd.DataFrame(data=resultados)
resultados.columns=['iter', 'Beta_i', 'Y1', 'Y2','Y3','Beta_i+1']

In [7]:
resultados

,iter,Beta_i,Y1,Y2,Y3,Beta_i+1
0,0.0,0.000000,-2.222222,-0.888889,1.777778,2.981424
1,1.0,2.981424,-3.063161,-0.926072,2.564507,4.100886
2,2.0,4.100886,-3.551887,-0.919323,2.979470,4.726341
3,3.0,4.726341,-3.858955,-0.899605,3.235906,5.115849
4,4.0,5.115849,-4.063492,-0.880982,3.403902,5.373513
